In [1]:
import pandas as pd
import numpy as np
# Load engineered dataset
df_model = pd.read_csv("tcs_features_5d.csv")

print(df_model.head())
print(df_model.shape)


         Date    Close    Open     High      Low   Volume    ret_1d    ret_5d  \
0  2019-10-29  2194.85  2121.5  2201.70  2121.50  5064173  0.037411  0.069928   
1  2019-10-30  2252.80  2225.0  2260.00  2176.85  5195136  0.026403  0.088257   
2  2019-10-31  2269.65  2250.0  2284.95  2237.00  4879261  0.007480  0.090130   
3  2019-11-01  2200.90  2264.0  2275.00  2195.00  4172620 -0.030291  0.035742   
4  2019-11-04  2193.95  2207.0  2229.20  2180.90  2646411 -0.003158  0.036985   

      ma_10  ma_dist_10  volatility_10d  vol_chg_1d  vol_ma_10  vol_ratio_10  \
0  2081.100    0.054659        0.013250   21.565504  3214421.6      1.575454   
1  2102.650    0.071410        0.014421    0.025861  3507804.2      1.481022   
2  2124.975    0.068083        0.014319   -0.060802  3692322.1      1.321461   
3  2141.970    0.027512        0.018654   -0.144825  3727706.7      1.119353   
4  2155.630    0.017777        0.018892   -0.365768  3683557.7      0.718439   

   target_5d  
0   0.003189  
1 

In [2]:
FEATURES = [
    "ret_1d",
    "ret_5d",
    "ma_dist_10",
    "volatility_10d",
    "vol_chg_1d",
    "vol_ratio_10"
]

X = df_model[FEATURES]
y = df_model["target_5d"]


In [3]:
split_idx = int(len(df_model) * 0.8)

X_train = X.iloc[:split_idx]
X_test  = X.iloc[split_idx:]

y_train = y.iloc[:split_idx]
y_test  = y.iloc[split_idx:]


In [4]:
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)


(980, 6) (246, 6)
(980,) (246,)


In [5]:
pip install xgboost


   ---------------------------------------- 0.0/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.3/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.3/101.7 MB ? eta -:--:--
   ---------------------------------------- 0.5/101.7 MB 829.2 kB/s eta 0:02:02
   ---------------------------------------- 0.5/101.7 MB 829.2 kB/s eta 0:02:02
   ---------------------------------------- 0.8/101.7 MB 617.6 kB/s eta 0:02:44
   ---------------------------------------- 1.0/101.7 MB 687.5 kB/s eta 0:02:27
   ---------------------------------------- 1.0/101.7 MB 687.5 kB/s eta 0:02:27
    --------------------------------------- 1.3/101.7 MB 737.7 kB/s eta 0:02:17
    --------------------------------------- 1.6/101.7 MB 774.0 kB/s eta 0:02:10
    --------------------------------------- 1.6/101.7 MB 774.0 kB/s eta 0:02:10
    -

In [5]:
from xgboost import XGBRegressor


In [6]:
xgb = XGBRegressor(
    n_estimators=400,
    learning_rate=0.05,
    max_depth=3,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_alpha=0.1,
    reg_lambda=1.0,
    random_state=42,
    objective="reg:squarederror",
    n_jobs=-1
)


In [7]:
xgb.fit(X_train, y_train)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.05, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=400,
             n_jobs=-1, num_parallel_tree=None, ...)

In [8]:
xgb_preds = xgb.predict(X_test)

print(xgb_preds[:10])


[ 0.01331068  0.00895803 -0.00023298  0.00609496 -0.00028641 -0.00558181
  0.00304083 -0.00394361 -0.00645074  0.00533865]


In [9]:
from sklearn.metrics import mean_absolute_error

xgb_mae = mean_absolute_error(y_test, xgb_preds)
xgb_dir_acc = np.mean(
    np.sign(xgb_preds) == np.sign(y_test)
)

print("XGB MAE:", xgb_mae)
print("XGB Directional Accuracy:", xgb_dir_acc)


XGB MAE: 0.024609626090395343
XGB Directional Accuracy: 0.5121951219512195
